In [17]:
#import statements
import pandas as pd 
import numpy as np 
import tensorflow as tf 
from pathlib import Path
import os
from sklearn.model_selection import train_test_split
from helper_methods import print_schema, encode_numeric_zscore, encode_text_dummy, print_column, encode_text_index, to_xy
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn import metrics
import matplotlib.pyplot as plt 

### Setup Environment
Create folders for putting test output in

In [18]:
base_path=os.path.join(os.getcwd(), 'test-output/')

iteration='iteration-0'
full_path = os.path.join(base_path, iteration)

try:
    os.mkdir(base_path)
    os.mkdir(full_path)
except Exception as e:
    print(f"{e}")

[WinError 183] Cannot create a file when that file already exists: 'c:\\Users\\User\\csc180\\network-traffic-analyzer\\test-output/'


In [19]:
#read from train/test datasets
training_dataframe = pd.read_csv(os.path.join(os.getcwd(), 'data/UNSW_NB15_training-set.csv'))
testing_dataframe = pd.read_csv(os.path.join(os.getcwd(), 'data/UNSW_NB15_testing-set.csv'))

In [20]:
#drop all entries that have a missing value anywhere in the set
training_dataframe = training_dataframe.replace('-', np.nan).dropna()
testing_dataframe = testing_dataframe.replace('-', np.nan).dropna()


In [21]:
testing_dataframe.shape

(35179, 45)

In [22]:
#get one instance of each value from each dataframe
training_unique = training_dataframe["proto"].unique()
testing_unique = testing_dataframe["proto"].unique()

#get only values from each numpy array
common_values = np.intersect1d(training_unique, testing_unique)

#make a filtered dataframe containing only values in common_values
training_dataframe = training_dataframe[training_dataframe['proto'].isin(common_values)]
testing_dataframe = testing_dataframe[testing_dataframe['proto'].isin(common_values)]

#same for service
training_unique = training_dataframe["service"].unique()
testing_unique = testing_dataframe["service"].unique()
common_values = np.intersect1d(training_unique, testing_unique)
training_dataframe = training_dataframe[training_dataframe['service'].isin(common_values)]
testing_dataframe = testing_dataframe[testing_dataframe['service'].isin(common_values)]

#same for state
training_unique = training_dataframe["state"].unique()
testing_unique = testing_dataframe["state"].unique()
common_values = np.intersect1d(training_unique, testing_unique)
training_dataframe = training_dataframe[training_dataframe['state'].isin(common_values)]
testing_dataframe = testing_dataframe[testing_dataframe['state'].isin(common_values)]


In [23]:
testing_dataframe.shape

(35178, 45)

In [24]:
print_schema(training_dataframe)
print_schema(testing_dataframe)

~~~~~~dataframe schema~~~~~~
Dataframe shape: (81159, 45) | Dataframe length: 81159
Column labels: 
Index(['id', 'dur', 'proto', 'service', 'state', 'spkts', 'dpkts', 'sbytes',
       'dbytes', 'rate', 'sttl', 'dttl', 'sload', 'dload', 'sloss', 'dloss',
       'sinpkt', 'dinpkt', 'sjit', 'djit', 'swin', 'stcpb', 'dtcpb', 'dwin',
       'tcprtt', 'synack', 'ackdat', 'smean', 'dmean', 'trans_depth',
       'response_body_len', 'ct_srv_src', 'ct_state_ttl', 'ct_dst_ltm',
       'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm',
       'is_ftp_login', 'ct_ftp_cmd', 'ct_flw_http_mthd', 'ct_src_ltm',
       'ct_srv_dst', 'is_sm_ips_ports', 'attack_cat', 'label'],
      dtype='object')
Dataframe head: 
    id       dur proto service state  spkts  dpkts  sbytes  dbytes  \
3    4  1.681642   tcp     ftp   FIN     12     12     628     770   
11  12  2.093085   tcp    smtp   FIN     62     28   56329    2212   
15  16  0.000002   udp    snmp   INT      2      0     138       0   
17  18  

### Encode Data

One-hot encode relevant rows


In [25]:
# one-hot encode non-y features of training dataset
encode_text_dummy(training_dataframe, 'proto')
encode_text_dummy(training_dataframe, 'service')
encode_text_dummy(training_dataframe, 'state')

# one-hot encode attack-category column (y)
encode_text_index(training_dataframe, 'attack_cat')
# one-hot encode non-y features of testing dataset
encode_text_dummy(testing_dataframe, 'proto')
encode_text_dummy(testing_dataframe, 'service')
encode_text_dummy(testing_dataframe, 'state')

# one-hot encode attack-category column (y)
encode_text_index(testing_dataframe, 'attack_cat')


array(['Backdoor', 'DoS', 'Exploits', 'Fuzzers', 'Generic', 'Normal',
       'Reconnaissance', 'Worms'], dtype=object)

In [26]:
print_schema(training_dataframe)
print_schema(testing_dataframe)


~~~~~~dataframe schema~~~~~~
Dataframe shape: (81159, 60) | Dataframe length: 81159
Column labels: 
Index(['id', 'dur', 'spkts', 'dpkts', 'sbytes', 'dbytes', 'rate', 'sttl',
       'dttl', 'sload', 'dload', 'sloss', 'dloss', 'sinpkt', 'dinpkt', 'sjit',
       'djit', 'swin', 'stcpb', 'dtcpb', 'dwin', 'tcprtt', 'synack', 'ackdat',
       'smean', 'dmean', 'trans_depth', 'response_body_len', 'ct_srv_src',
       'ct_state_ttl', 'ct_dst_ltm', 'ct_src_dport_ltm', 'ct_dst_sport_ltm',
       'ct_dst_src_ltm', 'is_ftp_login', 'ct_ftp_cmd', 'ct_flw_http_mthd',
       'ct_src_ltm', 'ct_srv_dst', 'is_sm_ips_ports', 'attack_cat', 'label',
       'proto-tcp', 'proto-udp', 'service-dhcp', 'service-dns', 'service-ftp',
       'service-ftp-data', 'service-http', 'service-irc', 'service-pop3',
       'service-radius', 'service-smtp', 'service-snmp', 'service-ssh',
       'service-ssl', 'state-CON', 'state-FIN', 'state-INT', 'state-REQ'],
      dtype='object')
Dataframe head: 
    id       dur  spkts  

Z-Score relevant rows

In [27]:
for column in training_dataframe.columns:
    if column not in ['service', 'srcip', 'dstip', 'proto', 'state','service', 'attack_cat', 'label', 'is_ftp_login', 'is_sm_ips_ports', 'Stime', 'Ltime']:
        encode_numeric_zscore(training_dataframe, column)
for column in testing_dataframe.columns:
    if column not in ['service', 'srcip', 'dstip', 'proto', 'state','service', 'attack_cat', 'label', 'is_ftp_login', 'is_sm_ips_ports', 'Stime', 'Ltime']:
        encode_numeric_zscore(testing_dataframe, column)


In [28]:
print_schema(training_dataframe)
print_schema(testing_dataframe)

~~~~~~dataframe schema~~~~~~
Dataframe shape: (81159, 60) | Dataframe length: 81159
Column labels: 
Index(['id', 'dur', 'spkts', 'dpkts', 'sbytes', 'dbytes', 'rate', 'sttl',
       'dttl', 'sload', 'dload', 'sloss', 'dloss', 'sinpkt', 'dinpkt', 'sjit',
       'djit', 'swin', 'stcpb', 'dtcpb', 'dwin', 'tcprtt', 'synack', 'ackdat',
       'smean', 'dmean', 'trans_depth', 'response_body_len', 'ct_srv_src',
       'ct_state_ttl', 'ct_dst_ltm', 'ct_src_dport_ltm', 'ct_dst_sport_ltm',
       'ct_dst_src_ltm', 'is_ftp_login', 'ct_ftp_cmd', 'ct_flw_http_mthd',
       'ct_src_ltm', 'ct_srv_dst', 'is_sm_ips_ports', 'attack_cat', 'label',
       'proto-tcp', 'proto-udp', 'service-dhcp', 'service-dns', 'service-ftp',
       'service-ftp-data', 'service-http', 'service-irc', 'service-pop3',
       'service-radius', 'service-smtp', 'service-snmp', 'service-ssh',
       'service-ssl', 'state-CON', 'state-FIN', 'state-INT', 'state-REQ'],
      dtype='object')
Dataframe head: 
          id       dur   

In [29]:
x_train,y_train=to_xy(training_dataframe, 'label')
x_test, y_test=to_xy(testing_dataframe, 'label')

### Create Fully Connected Model
Design the model's architecture. 

In [ ]:
# TODO implement a more robust training setup, implement model tabulation 

checkpointer = ModelCheckpoint(filepath=os.path.join(full_path, "best_weights.keras"), verbose=0, save_best_only=True) # save best model
for i in range(5):
        print(f"training model: {i}")
        model = Sequential()
        model.add(Dense(512, input_dim=x_train.shape[1], activation='relu'))
        model.add(Dense(256, activation='relu'))
        model.add(Dropout(0.3))
        model.add(Dense(128, activation='relu'))
        model.add(Dropout(0.3))
        model.add(Dense(64, activation='relu'))
        model.add(Dropout(0.3))
        model.add(Dense(32, activation='relu'))
        model.add(Dropout(0.3))
        model.add(Dense(16, activation='relu'))
        model.add(Dense(8, activation='relu'))
        # model.add(Dense(4, activation='relu'))
        model.add(Dense(2, activation='sigmoid')) # single output w/o activation function b/c regression
        model.compile(loss='binary_crossentropy', optimizer=SGD(learning_rate=0.01, momentum=0.01))
        monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=10, verbose=2, mode='min', restore_best_weights=True)
        history = model.fit(x_train, y_train, validation_data=(x_test, y_test),batch_size=64, callbacks=[monitor, checkpointer], verbose=2, epochs=100)

training model: 0


c:\Users\User\anaconda3\envs\lab01\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100


ValueError: Could not interpret loss identifier: binary_cross_entropy